<a href="https://colab.research.google.com/github/lubnakhan-Github-Account/agentic_class_work/blob/main/Hooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***RunHooks And AgentHooks***

In [2]:
!pip install -qU openai-agents

import os
from google.colab import userdata

import nest_asyncio
nest_asyncio.apply()

api_key=userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

from agents import Agent, RunHooks, Runner,AgentHooks,RunContextWrapper,function_tool,enable_verbose_stdout_logging
from dotenv import load_dotenv
from typing import Any

# -------------------------------Run Hook--------------------
class  Run_Hook(RunHooks):

  async def on_agent_start(self, context:RunContextWrapper, agent:Agent,)->None:
      """this hook run before agent"""
      print("⚓🔧 1: agent_start  event start before the the agent start.")

  async def on_agent_end(self, context:RunContextWrapper, agent:Agent, output:Any)->None:
      print(" ⚓🔧 2:agent_end event start after the   final output.")

  async def on_handoff(self, context:RunContextWrapper, from_agent:Agent, to_agent:Agent)-> None:
    #  return await super().on_handoff(context, from_agent, to_agent)
       print("⚓🔧 3: agent start again before handoff.")

  async def on_tool_start(self,context:RunContextWrapper, agent:Agent,tool)->None:
       print(" ⚓🔧 4: tool_start event start before the tool start. ")

  async def on_tool_end(self, context:RunContextWrapper, agent:Agent,tool, result:str)->None:
      print("⚓🔧 5: tool_end event start after the tool  end.")

my_run_hook= Run_Hook()
# ------------Agent Hook ----------------------
class AgentHookClass(AgentHooks):

    async def on_start(self, context:RunContextWrapper, agent:Agent)->None:
        print("🤖  1:on start event start before agent invoke.")

    async def on_end(self, context:RunContextWrapper, agent:Agent, output:Any)->None:
        print("🤖 2:on end event start after agent end.")

    async def on_handoff(self, context, agent, source):
        print("🤖 3:on handoff event start before handoff invoke.")

    async def on_tool_start(self, context:RunContextWrapper, agent:Agent, tool)->None:
          print("🤖 4:on tool start event start before tool invoke")
    async def on_tool_end(self, context, agent, tool, result):
        print("🤖 5:on tool end event start after tool end.")

#--------------------------------------------------------------------------
# ------------Agent Hook ----------------------
class SecAgentHookClass(AgentHooks):

    async def on_start(self, context:RunContextWrapper, agent:Agent)->None:
        print("✅  1:on start event start before agent invoke.")

    async def on_end(self, context:RunContextWrapper, agent:Agent, output:Any)->None:
        print("✅ 2:on end event start after agent end.")

    async def on_handoff(self, context, agent, source):
        print("✅ 3:on handoff event start before handoff invoke.")

    async def on_tool_start(self, context:RunContextWrapper, agent:Agent, tool)->None:
          print("✅ 4:on tool start event start before tool invoke")
    async def on_tool_end(self, context, agent, tool, result):
        print("✅ 5:on tool end event start after tool end.")

#--------------------------------------------------------------------------
@function_tool
def weather_tool():
    return f"weather is mostly 🌩 cloudy."
# -----------------------------------------------------------------------
my_handoff_agent=Agent(
    name="my_handoff_agent",
    instructions="You are a helpful assistent.",
    model="gpt-4.1-mini",
    hooks=SecAgentHookClass(),
)
# ----------------------------------------------------------------
agent= Agent(
    name="triage_agent",
    instructions="You are a helpful assistent.",
    model="gpt-4.1-mini",
    hooks=AgentHookClass(),
    handoffs=[my_handoff_agent],
    tools=[weather_tool]

)

result= Runner.run_sync(agent,"call  my_handoff_agent, call weather_tool, ",hooks=my_run_hook)
print(result.final_output)

⚓🔧 1: agent_start  event start before the the agent start.
🤖  1:on start event start before agent invoke.
 ⚓🔧 4: tool_start event start before the tool start. 
🤖 4:on tool start event start before tool invoke
⚓🔧 5: tool_end event start after the tool  end.
🤖 5:on tool end event start after tool end.
⚓🔧 3: agent start again before handoff.
🤖 3:on handoff event start before handoff invoke.
⚓🔧 1: agent_start  event start before the the agent start.
✅  1:on start event start before agent invoke.
 ⚓🔧 2:agent_end event start after the   final output.
✅ 2:on end event start after agent end.
I've checked the weather for you—it's mostly cloudy with some thunderstorms. Also, I am calling your handoff agent now. How can I assist you further?
